## 0. Load imports 

In [141]:
## imports
import pandas as pd
import numpy as np


## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 0. Load data

In [143]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
dc_crim_2020.head()
dc_crim_2020.shape
dc_crim_2020.info()


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.044182,38.905654,11139598,2020/08/24 04:00:00+00,MIDNIGHT,OTHERS,HOMICIDE,1900 - 1999 BLOCK OF M STREET NW,396168.0,137533.0,...,010700 1,10700.0,Precinct 17,38.905646,-77.044180,GOLDEN TRIANGLE,2011/09/23 06:40:00+00,NaN,594430168,NaN
1,-77.023981,38.900345,5370,2020/01/17 02:39:05+00,EVENING,OTHERS,THEFT F/AUTO,800 - 899 BLOCK OF 9TH STREET NW,397920.0,136943.0,...,005802 5,5802.0,Precinct 129,38.900337,-77.023979,DOWNTOWN,2020/01/11 02:48:51+00,2020/01/17 02:48:54+00,594430173,NaN
2,-76.972651,38.857491,10251445,2020/09/11 04:00:00+00,MIDNIGHT,GUN,HOMICIDE,2300 - 2399 BLOCK OF AINGER PLACE SE,402374.0,132186.0,...,007502 1,7502.0,Precinct 134,38.857483,-76.972648,NaN,2010/09/02 04:00:00+00,2010/09/02 04:00:00+00,594430178,NaN
3,-76.972226,38.854086,10147537,2020/12/21 05:00:00+00,MIDNIGHT,OTHERS,HOMICIDE,2300 - 2499 BLOCK OF HARTFORD STREET SE,402411.0,131808.0,...,007408 2,7408.0,Precinct 115,38.854078,-76.972223,NaN,2010/10/10 02:00:00+00,NaN,594431372,NaN
4,-77.001753,38.821632,18048023,2020/10/21 04:00:00+00,MIDNIGHT,OTHERS,HOMICIDE,4600 - 4799 BLOCK OF SOUTH CAPITOL STREET,399848.0,128205.0,...,009811 1,9811.0,Precinct 125,38.821624,-77.001750,NaN,2018/03/26 08:40:06+00,2018/03/26 10:04:44+00,594431711,NaN


(27931, 25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27931 entries, 0 to 27930
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     27931 non-null  float64
 1   Y                     27931 non-null  float64
 2   CCN                   27931 non-null  int64  
 3   REPORT_DAT            27931 non-null  object 
 4   SHIFT                 27931 non-null  object 
 5   METHOD                27931 non-null  object 
 6   OFFENSE               27931 non-null  object 
 7   BLOCK                 27931 non-null  object 
 8   XBLOCK                27931 non-null  float64
 9   YBLOCK                27931 non-null  float64
 10  WARD                  27931 non-null  int64  
 11  ANC                   27931 non-null  object 
 12  DISTRICT              27902 non-null  float64
 13  PSA                   27896 non-null  float64
 14  NEIGHBORHOOD_CLUSTER  27931 non-null  object 
 15  BLOCK_GROUP        

#### Warm-up: Ways to add 100 to column X

In [145]:
%%time
df.X + 100

CPU times: user 769 μs, sys: 336 μs, total: 1.1 ms
Wall time: 1.07 ms


0        22.955818
1        22.976019
2        23.027349
3        23.027774
4        22.998247
           ...    
27926    22.951183
27927    22.996471
27928    22.973471
27929    23.006236
27930    22.952276
Name: X, Length: 27931, dtype: float64

## 1. Aggregation

### Example of grouping by one variable and doing one aggregation 

**Task**: find and print the number of unique offense types (`OFFENSE`) by ward (`WARD`)

In [147]:
df.groupby("WARD")["OFFENSE"].nunique()

WARD
1    9
2    9
3    8
4    8
5    9
6    9
7    9
8    9
Name: OFFENSE, dtype: int64

### Example of grouping by one variable and providing two summaries of the same variable

**Task**: previous showed number of offenses by ward but want to find out content of offenses in each ward; create an aggregation that summarizes both the number of unique offenses by ward and what those offenses are -- for instance, by pasting the unique offenses in that ward separated by the ";" (e.g., Theft; Burglary;...)

*Hint*: you can use the join command to paste together a list separated by some delimiter. 
    - The syntax, if we are using the comma delimiter, is: ", ".join(nameoflist)

In [150]:
df1 = df.groupby("WARD")["OFFENSE"].unique()

def count_fx(x):
    return len(x)

def join_text(x):
    return "; ".join(x)

new_list = []
for i, l in list( df1.items() ):
    new_list.append( join_text(l))
    
df1.apply(count_fx)
df2 = df1.reset_index()
df2["uniq_off_by_ward"] = new_list
df2

WARD
1    9
2    9
3    8
4    8
5    9
6    9
7    9
8    9
Name: OFFENSE, dtype: int64

,WARD,OFFENSE,uniq_off_by_ward
0,1,"[THEFT/OTHER, ASSAULT W/DANGEROUS WEAPON, ROBB...",THEFT/OTHER; ASSAULT W/DANGEROUS WEAPON; ROBBE...
1,2,"[HOMICIDE, THEFT F/AUTO, THEFT/OTHER, MOTOR VE...",HOMICIDE; THEFT F/AUTO; THEFT/OTHER; MOTOR VEH...
2,3,"[THEFT F/AUTO, THEFT/OTHER, MOTOR VEHICLE THEF...",THEFT F/AUTO; THEFT/OTHER; MOTOR VEHICLE THEFT...
3,4,"[THEFT F/AUTO, THEFT/OTHER, HOMICIDE, MOTOR VE...",THEFT F/AUTO; THEFT/OTHER; HOMICIDE; MOTOR VEH...
4,5,"[THEFT/OTHER, THEFT F/AUTO, MOTOR VEHICLE THEF...",THEFT/OTHER; THEFT F/AUTO; MOTOR VEHICLE THEFT...
5,6,"[ROBBERY, THEFT/OTHER, THEFT F/AUTO, MOTOR VEH...",ROBBERY; THEFT/OTHER; THEFT F/AUTO; MOTOR VEHI...
6,7,"[THEFT F/AUTO, ROBBERY, ASSAULT W/DANGEROUS WE...",THEFT F/AUTO; ROBBERY; ASSAULT W/DANGEROUS WEA...
7,8,"[HOMICIDE, THEFT/OTHER, THEFT F/AUTO, MOTOR VE...",HOMICIDE; THEFT/OTHER; THEFT F/AUTO; MOTOR VEH...


### Example of grouping by two variables 

**Task**: group by ward (`WARD`) and police shift (`SHIFT`) and find the offense that is most common in that ward and shift

In [153]:
df.groupby(["WARD", "SHIFT"]).agg(
    {
        "OFFENSE": lambda x: x.mode()
    }
).reset_index()

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


## Summarizing over all rows or all columns (without grouping)

We can also use the `apply` function to summarize rows or columns efficiently

**Task**: find the mean lat and longitude in one line of code

In [157]:
df[["X","Y"]].apply(np.mean, axis=0)

X   -77.007049
Y    38.907618
dtype: float64

## 2. Creating new columns/transforming their type 

### Simple filtering

`np.where` is in the numpy package (aliased as `np`) and operates similar to `ifelse` in R

But there is a built in pandas way to emulate this.

**Task**: create an indicator `is_theft` for any offense that contains the word "THEFT"
    
**Task**: create an indicator `is_theft_notmotor` for any offense that contains the word "THEFT" but does not contain the word "MOTOR"

In [221]:
df["is_theft"] = df.OFFENSE.str.contains("THEFT")
print(df.is_theft)

df["is_theft_notmotor"] = df.OFFENSE.str.contains("THEFT") & ~df.OFFENSE.str.contains("MOTOR")

0        False
1         True
2        False
3        False
4        False
         ...  
27926     True
27927     True
27928     True
27929     True
27930     True
Name: is_theft, Length: 27931, dtype: bool


### np.select

**Task**: create a new variable, `offense_summary`, where you:
        
- Recode theft offenses that use a gun or knife as the method (`METHOD`) as: violent theft
- Recode non-theft offenses that use a gun or knife as the method as: violent other
- Recode all other as non-violent 

In [227]:
method = ["violent_theft", "violent_other"]

conditions = [df.METHOD.isin(["GUN", "KNIFE"]) & df.OFFENSE.str.contains("THEFT"),
              df.METHOD.isin(["GUN", "KNIFE"]) & ~df.OFFENSE.str.contains("THEFT")
             ]

df["offense_summary"] = np.select(conditions, method, default = "non-violent")
df.offense_summary.head()

df["offense_summary"].value_counts()

0      non-violent
1      non-violent
2    violent_other
3      non-violent
4      non-violent
Name: offense_summary, dtype: object

offense_summary
non-violent      25256
violent_other     2640
violent_theft       35
Name: count, dtype: int64

In [119]:
df.METHOD.head()

0    OTHERS
1    OTHERS
2       GUN
3    OTHERS
4    OTHERS
Name: METHOD, dtype: object

### map.recode

**Task**: recode shifts that are MIDNIGHT or EVENING as "nighttime"; code other shift to daytime

In [163]:
df.SHIFT

0        MIDNIGHT
1         EVENING
2        MIDNIGHT
3        MIDNIGHT
4        MIDNIGHT
           ...   
27926    MIDNIGHT
27927         DAY
27928         DAY
27929     EVENING
27930     EVENING
Name: SHIFT, Length: 27931, dtype: object

In [167]:
time = ["DAY", "EVENING", "MIDNIGHT"]

new = ["daytime", "nighttime", "nighttime"]

conversion_dict = dict(zip(time, new))

df["NEW_SHIFT"] = df.SHIFT.map(conversion_dict)

df.NEW_SHIFT

# df.SHIFT.apply(lambda x: conversion_dict[x])

0        nighttime
1        nighttime
2        nighttime
3        nighttime
4        nighttime
           ...    
27926    nighttime
27927      daytime
27928      daytime
27929    nighttime
27930    nighttime
Name: NEW_SHIFT, Length: 27931, dtype: object

### Using built-in pandas methods

Another way of creating variables, used either alone or in combination with np.where and np.select is to use pandas built in `str` methods

Basic structure is: `df['namestringcol'].str.someoperation`

**Task**: using a `str` method, create a new variable--`OFFENSE_NOSP`--that replaces spaces in the `OFFENSE` column with underscores

In [173]:
df["OFFENSE_NOSP"] = df["OFFENSE"].str.replace(" ", "_")

df.OFFENSE_NOSP.head()
df.OFFENSE.head()

0        HOMICIDE
1    THEFT_F/AUTO
2        HOMICIDE
3        HOMICIDE
4        HOMICIDE
Name: OFFENSE_NOSP, dtype: object

0        HOMICIDE
1    THEFT F/AUTO
2        HOMICIDE
3        HOMICIDE
4        HOMICIDE
Name: OFFENSE, dtype: object

### Transforming column types

**Task**: check the type of the `START_DATE` column

In [181]:
print(df["START_DATE"].dtype)

object


**Task**: recast the `START_DATE` column to datetime, calling this `START_DATE_CLEAN`; coerce errors rather than cleaning the string; check the type of that new col

In [185]:
df["START_DATE_CLEAN"] = pd.to_datetime(df["START_DATE"], errors='coerce')

print(df["START_DATE_CLEAN"].dtype)

datetime64[ns, UTC]


**Task:** print the min and max of `START_DATE_CLEAN`. What happens if you try to do this with `START_DATE`?

In [193]:
print(df["START_DATE_CLEAN"].min())
print(df["START_DATE_CLEAN"].max())
print(df["START_DATE"].min())
print(df["START_DATE"].max())

1940-02-17 00:00:16+00:00
2021-01-01 02:30:21+00:00
1940/02/17 00:00:16+00
2021/01/01 02:30:21+00


## 3. Row and column filtering 

### Row subsetting

**Task**: filter to crime reports about theft using the `is_theft` definition and that are in ward 3

In [203]:
filtered_df = df[(df["is_theft"]) & (df['WARD'] == 3)]

print(filtered_df[["WARD", "is_theft"]])



       WARD  is_theft
8         3      True
12        3      True
13        3      True
22        3      True
39        3      True
...     ...       ...
27801     3      True
27828     3      True
27907     3      True
27909     3      True
27914     3      True

[1522 rows x 2 columns]


### Column subsetting


**Task**: select two columns--`START_DATE` and `END_DATE`--and print the head of the dataframe

In [205]:
df_select = df[["START_DATE", "END_DATE"]]

print(df_select.head())



               START_DATE                END_DATE
0  2011/09/23 06:40:00+00                     NaN
1  2020/01/11 02:48:51+00  2020/01/17 02:48:54+00
2  2010/09/02 04:00:00+00  2010/09/02 04:00:00+00
3  2010/10/10 02:00:00+00                     NaN
4  2018/03/26 08:40:06+00  2018/03/26 10:04:44+00


**Task**: use list comprehension to automate this a bit and select all columns with the word "DATE" in the name; print the head of the dataframe

In [207]:
date_columns = [c for c in df.columns if "DATE" in c]

print(df[date_columns].head())



               START_DATE                END_DATE          START_DATE_CLEAN
0  2011/09/23 06:40:00+00                     NaN 2011-09-23 06:40:00+00:00
1  2020/01/11 02:48:51+00  2020/01/17 02:48:54+00 2020-01-11 02:48:51+00:00
2  2010/09/02 04:00:00+00  2010/09/02 04:00:00+00 2010-09-02 04:00:00+00:00
3  2010/10/10 02:00:00+00                     NaN 2010-10-10 02:00:00+00:00
4  2018/03/26 08:40:06+00  2018/03/26 10:04:44+00 2018-03-26 08:40:06+00:00


**Task**: filter reports of crime in `START_DATE_CLEAN` before May 2020 and that are located in ward 1; select the ward and `START_DATE` column and print a random sample of rows

In [211]:
df_tech = (df["START_DATE_CLEAN"] < "2020-05-01") & (df["WARD"] == 1)

row_select = df[["START_DATE", "WARD"]]


print(row_select[df_tech].head())


                START_DATE  WARD
9   2020/01/03 04:20:19+00     1
16  2020/01/08 20:44:58+00     1
25  2020/02/21 17:15:39+00     1
60  2020/01/28 15:23:51+00     1
80  2020/02/18 23:09:54+00     1
